In [1]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import datetime as dt
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [2]:

df = pd.read_csv('/kaggle/input/cardiovascular-disease-dataset/cardio_train.csv', delimiter=';')


In [3]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
# Converting the age in days to years
df['age'] = df['age']/365
df['age'] = df['age'].round(2)
# Converting the height in cm to ft
df['height'] = df['height']*0.032808399
df['height'] = df['height'].round(2)
df = df.rename(columns={'age': 'age_years', 'height': 'height_ft'})

df = df.drop("id", axis=1)

In [5]:
df['cardio'].value_counts(normalize=True)
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)


In [6]:
# Separate instances belonging to class 0 and class 1 in the train dataset


In [7]:
train_data['cardio'].value_counts(normalize=True)


0    0.501224
1    0.498776
Name: cardio, dtype: float64

In [8]:
train_data = train_data.reset_index(drop=True)

X_train_data = train_data.drop("cardio", axis=True)

y_train_data = train_data["cardio"]

In [9]:
X_train_data.head()


,age_years,gender,height_ft,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,55.94,1,5.25,64.0,120,90,3,1,0,0,1
1,62.16,2,5.48,65.0,120,80,3,3,0,0,0
2,62.42,1,5.25,66.0,120,90,1,1,0,0,1
3,62.05,1,5.35,55.0,125,90,3,1,0,0,1
4,59.48,1,5.18,85.0,150,80,3,1,0,0,1


In [10]:
test_data = test_data.reset_index(drop=True)

X_test_data = test_data.drop("cardio", axis=True)

y_test_data = test_data["cardio"]

In [11]:
X_test_data.head()


,age_years,gender,height_ft,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,59.64,1,5.12,64.0,140,80,2,1,0,0,1
1,59.93,1,5.58,85.0,160,90,1,1,0,0,1
2,63.75,1,4.95,90.0,130,80,1,1,0,0,1
3,54.08,1,5.22,97.0,120,80,1,1,0,0,1
4,50.40,1,5.38,68.0,120,80,1,1,0,0,1


In [12]:
model_IF = IsolationForest(contamination=float(0.3),random_state=42)

# Create and fit the Isolation Forest model
model_IF.fit(X_train_data)

# Generate outlier scores for instances
outlier_scores = model_IF.decision_function(X_test_data)

# Define a threshold to classify instances as anomalies or normal data
threshold = 0.0

# Convert outlier scores to binary labels based on the threshold
predictions = [1 if score < threshold else 0 for score in outlier_scores]

# Calculate precision, recall, and F1-score
print(classification_report(y_test_data, predictions))

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

           0       0.55      0.77      0.64     10461
           1       0.62      0.36      0.46     10539

    accuracy                           0.57     21000
   macro avg       0.58      0.57      0.55     21000
weighted avg       0.58      0.57      0.55     21000

